In [ ]:
1.

In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


movies_df = pd.read_csv("./movies.csv",na_values='NA', keep_default_na=False , encoding = "ISO-8859-1")

# Seleccionar las columnas numéricas para el análisis
features_num = [
    'budget', 'revenue', 'runtime', 'popularity', 'voteAvg', 'voteCount',
    'genresAmount', 'productionCoAmount', 'productionCountriesAmount', 'actorsAmount'
]

# Preparar el preprocesamiento para las variables numéricas

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar valores faltantes con la media
    ('scaler', StandardScaler())  # Escalar las variables
])

# Aplicar el preprocesamiento a las variables numéricas
X_preprocessed = numeric_transformer.fit_transform(movies_df[features_num])
X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=features_num)
# Guardar el DataFrame preprocesado en un archivo CSV
X_preprocessed_df.to_csv("./movies.csv", index=False)

print(X_preprocessed_df.head())


     budget   revenue   runtime  popularity   voteAvg  voteCount  \
0 -0.404342 -0.355681 -0.086728   -0.142920 -0.809539   0.274380   
1  0.055440 -0.303564  0.343289   -0.195656  0.007403  -0.442230   
2 -0.215020  4.744733  0.737472    0.226864  1.743405   5.887053   
3  2.029796  5.835649 -0.015059    0.387782  1.334934   5.628083   
4  0.975003  4.096481  1.490002    0.034071  2.049758   7.992433   

   genresAmount  productionCoAmount  productionCountriesAmount  actorsAmount  
0     -0.519953           -0.468349                  -0.250235     -0.057791  
1      0.344007           -0.078445                   0.078158     -0.058056  
2      0.344007           -0.468349                  -0.250235     -0.055667  
3     -0.519953           -0.858253                  -0.250235     -0.057818  
4      0.344007           -0.468349                  -0.250235     -0.056437  
